In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"evriliyasyahutami","key":"304b19e33d3c1fe20bf4465b01154e46"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

kaggle.json


In [3]:
!kaggle datasets download -d jillanisofttech/brain-stroke-dataset

Dataset URL: https://www.kaggle.com/datasets/jillanisofttech/brain-stroke-dataset
License(s): DbCL-1.0
  0% 0.00/47.2k [00:00<?, ?B/s]
100% 47.2k/47.2k [00:00<00:00, 46.1MB/s]


In [7]:
!unzip brain-stroke-dataset.zip -d brain-stroke-dataset
!ls brain-stroke-dataset

Archive:  brain-stroke-dataset.zip
  inflating: brain-stroke-dataset/brain_stroke.csv  
brain_stroke.csv


In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Concatenate

In [18]:
df = pd.read_csv("/content/brain-stroke-dataset/brain_stroke.csv")
df.head()

gender   age  hypertension  heart_disease ever_married      work_type  \
0    Male  67.0             0              1          Yes        Private   
1    Male  80.0             0              1          Yes        Private   
2  Female  49.0             0              0          Yes        Private   
3  Female  79.0             1              0          Yes  Self-employed   
4    Male  81.0             0              0          Yes        Private   

  Residence_type  avg_glucose_level   bmi   smoking_status  stroke  
0          Urban             228.69  36.6  formerly smoked       1  
1          Rural             105.92  32.5     never smoked       1  
2          Urban             171.23  34.4           smokes       1  
3          Rural             174.12  24.0     never smoked       1  
4          Urban             186.21  29.0  formerly smoked       1

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4981 entries, 0 to 4980
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             4981 non-null   object 
 1   age                4981 non-null   float64
 2   hypertension       4981 non-null   int64  
 3   heart_disease      4981 non-null   int64  
 4   ever_married       4981 non-null   object 
 5   work_type          4981 non-null   object 
 6   Residence_type     4981 non-null   object 
 7   avg_glucose_level  4981 non-null   float64
 8   bmi                4981 non-null   float64
 9   smoking_status     4981 non-null   object 
 10  stroke             4981 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 428.2+ KB


In [20]:
col_names = ['gender','age','hypertension','heart_disease','heart_disease','ever_married','work_type','Residence_type','avg_glucose_level','bmi','smoking_status','stroke']

for col in col_names:
    print (df[col].value_counts())

gender
Female    2907
Male      2074
Name: count, dtype: int64
age
78.00    102
57.00     92
54.00     85
51.00     84
79.00     84
        ... 
1.16       3
0.48       3
0.40       2
0.08       2
0.16       1
Name: count, Length: 104, dtype: int64
hypertension
0    4502
1     479
Name: count, dtype: int64
heart_disease
0    4706
1     275
Name: count, dtype: int64
heart_disease
0    4706
1     275
Name: count, dtype: int64
ever_married
Yes    3280
No     1701
Name: count, dtype: int64
work_type
Private          2860
Self-employed     804
children          673
Govt_job          644
Name: count, dtype: int64
Residence_type
Urban    2532
Rural    2449
Name: count, dtype: int64
avg_glucose_level
93.88     6
73.00     5
83.16     5
91.85     5
72.49     5
         ..
130.56    1
67.78     1
116.12    1
59.63     1
83.94     1
Name: count, Length: 3895, dtype: int64
bmi
28.7    42
28.4    41
27.3    38
27.7    37
26.7    37
        ..
48.4     1
48.2     1
46.6     1
40.6     1
14.9     1
N

In [21]:
numerical = []
catgcols =[]

for col in df.columns:
  if df[col].dtype=="float64":
    numerical.append(col)
  else:
    catgcols.append(col)

for col in df.columns:
  if col in numerical:
    df[col].fillna(df[col].median(), inplace=True)
  else:
    df[col].fillna(df[col].mode()[0], inplace=True)

In [22]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for col in catgcols:
    df[col] = le.fit_transform(df[col])

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4981 entries, 0 to 4980
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             4981 non-null   int64  
 1   age                4981 non-null   float64
 2   hypertension       4981 non-null   int64  
 3   heart_disease      4981 non-null   int64  
 4   ever_married       4981 non-null   int64  
 5   work_type          4981 non-null   int64  
 6   Residence_type     4981 non-null   int64  
 7   avg_glucose_level  4981 non-null   float64
 8   bmi                4981 non-null   float64
 9   smoking_status     4981 non-null   int64  
 10  stroke             4981 non-null   int64  
dtypes: float64(3), int64(8)
memory usage: 428.2 KB


In [24]:
df.drop(columns=['avg_glucose_level','bmi','smoking_status'], inplace=True)

In [25]:
X = df.drop("stroke",axis=1)
y = pd.get_dummies(df['stroke'])

In [26]:
from sklearn import preprocessing
col_names = list(X.columns)
s_scaler = preprocessing.StandardScaler()
X_df= s_scaler.fit_transform(X)
X_df = pd.DataFrame(X_df, columns=col_names)
X_df.describe().T

count          mean     std       min       25%       50%  \
gender          4981.0  1.711807e-17  1.0001 -0.844660 -0.844660 -0.844660   
age             4981.0  2.853012e-18  1.0001 -1.912575 -0.812863  0.069731   
hypertension    4981.0 -1.426506e-17  1.0001 -0.326186 -0.326186 -0.326186   
heart_disease   4981.0  2.567711e-17  1.0001 -0.241735 -0.241735 -0.241735   
ever_married    4981.0 -5.278073e-17  1.0001 -1.388624 -1.388624  0.720137   
work_type       4981.0  1.433639e-16  1.0001 -1.514212 -0.351534 -0.351534   
Residence_type  4981.0  2.853012e-18  1.0001 -1.016804 -1.016804  0.983473   

                     75%       max  
gender          1.183909  1.183909  
age             0.775806  1.702530  
hypertension   -0.326186  3.065738  
heart_disease  -0.241735  4.136753  
ever_married    0.720137  0.720137  
work_type       0.811143  1.973821  
Residence_type  0.983473  0.983473

In [27]:
X_train, X_test, y_train,y_test = train_test_split(X_df,y,test_size=0.25,random_state=7)

In [28]:
print(f"Shape of X_train is: {X_train.shape}")
print(f"Shape of y_train is: {y_train.shape}\n")
print(f"Shape of X_test is: {X_test.shape}")
print(f"Shape of y_test is: {y_test.shape}")

Shape of X_train is: (3735, 7)
Shape of y_train is: (3735, 2)

Shape of X_test is: (1246, 7)
Shape of y_test is: (1246, 2)


In [29]:
input_dim = X.shape[1]

In [30]:
model = Sequential()
model.add(Dense(8, input_dim = input_dim, kernel_initializer='normal', activation='relu'))
model.add(Dense(5, activation = "relu", kernel_initializer='normal'))
model.add(Dense(2, activation = 'sigmoid'))

In [31]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer = Adam(learning_rate = 0.001),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 64        
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 2)                 12        
                                                                 
Total params: 121 (484.00 Byte)
Trainable params: 121 (484.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
result = model.fit(X_train, y_train, epochs = 15, batch_size = 10)

Epoch 1/15
374/374 [==============================] - 2s 2ms/step - loss: 0.3670 - accuracy: 0.9494
Epoch 2/15
374/374 [==============================] - 1s 2ms/step - loss: 0.1667 - accuracy: 0.9502
Epoch 3/15
374/374 [==============================] - 1s 2ms/step - loss: 0.1600 - accuracy: 0.9502
Epoch 4/15
374/374 [==============================] - 1s 2ms/step - loss: 0.1575 - accuracy: 0.9502
Epoch 5/15
374/374 [==============================] - 1s 2ms/step - loss: 0.1565 - accuracy: 0.9502
Epoch 6/15
374/374 [==============================] - 1s 2ms/step - loss: 0.1551 - accuracy: 0.9502
Epoch 7/15
374/374 [==============================] - 1s 2ms/step - loss: 0.1548 - accuracy: 0.9502
Epoch 8/15
374/374 [==============================] - 1s 2ms/step - loss: 0.1546 - accuracy: 0.9502
Epoch 9/15
374/374 [==============================] - 1s 2ms/step - loss: 0.1540 - accuracy: 0.9502
Epoch 10/15
374/374 [==============================] - 1s 2ms/step - loss: 0.1543 - accuracy: 0.9502

In [34]:
y_pred =model.predict(X_test)
y_pred = (y_pred>0.5)

39/39 [==============================] - 0s 2ms/step


In [35]:
accuracy = accuracy_score(y_test,y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.9502407704654896


In [36]:
model.save('./brain.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [37]:
model = tf.keras.models.load_model('./brain.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
open("./brain.tflite", "wb").write(tflite_model)

2548